# 🚀 Introduction to MLflow Tracking

This script demonstrates the core capabilities of **MLflow Tracking**. Whenever we train a machine learning model, we want to record the exact conditions under which it was trained and how well it performed. Instead of relying on messy spreadsheets or console prints, MLflow acts as an automated "lab notebook" for our machine learning experiments.



### 🔍 What this script does:

1. **Sets up a Tracking Server:** Connects to a local SQLite database (`mlflow.db`) to safely store our experiment data instead of losing it when the script finishes.
2. **Creates an Experiment:** Groups our work under the folder name `Regression-Experiment`.
3. **Starts a "Run":** Opens a recording session (`linear_regression_baseline`) to track this specific attempt at training a model.
4. **Logs the following to MLflow:**
   * **Parameters (`log_param`):** The configuration choices we made (e.g., model type, random state). Think of these as the *inputs* of the experiment.
   * **Metrics (`log_metric`):** How well the model performed (e.g., RMSE, R-squared). Think of these as the *results* of the experiment.
   * **Artifacts (`log_artifact`):** Output files generated during training, such as the residual scatter plot we create using Matplotlib.
   * **The Model (`log_model`):** The actual trained Scikit-learn model, saved so it can be loaded later to make predictions without needing to be retrained.

### 🛠️ Instructions for Students:
Read through the comments in the code below to see exactly how these tracking functions are implemented, and then run the script to log your first experiment!

In [1]:
import os
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

In [ ]:

# Set the tracking URI to a local SQLite database. You can also use a remote server or a file-based store.
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Sets the active experiment. An "Experiment" is a container for multiple "Runs". 
# If "Regression-Experiment" doesn't exist, MLflow creates it.
mlflow.set_experiment("Regression-Experiment")

print("Connected to:", mlflow.get_tracking_uri())


2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/21 09:51:05 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/21 09:51:06 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/21 09:51:06 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/21 09:51:06 INFO mlflow.tracking.fluent: Experiment with name 'Regression-Experiment' does not exist. Creating a new experiment.


Connected to: sqlite:///mlflow.db


In [4]:
# Dataset
X, y = load_diabetes(return_X_y=True)

print("Dataset loaded. Shape of X:", X.shape, "Shape of y:", y.shape)
print("First 5 rows of X:\n", X[:5])
print("First 5 values of y:\n", y[:5])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Dataset loaded. Shape of X: (442, 10) Shape of y: (442,)
First 5 rows of X:
 [[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
  -0.04340085 -0.00259226  0.01990749 -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 -0.02632753 -0.00844872 -0.01916334
   0.07441156 -0.03949338 -0.06833155 -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 -0.00567042 -0.04559945 -0.03419447
  -0.03235593 -0.00259226  0.00286131 -0.02593034]
 [-0.08906294 -0.04464164 -0.01159501 -0.03665608  0.01219057  0.02499059
  -0.03603757  0.03430886  0.02268774 -0.00936191]
 [ 0.00538306 -0.04464164 -0.03638469  0.02187239  0.00393485  0.01559614
   0.00814208 -0.00259226 -0.03198764 -0.04664087]]
First 5 values of y:
 [151.  75. 141. 206. 135.]


In [6]:

# This is the core of the script. Everything indented inside this with block is executed as a single "Run".
with mlflow.start_run(run_name="linear_regression_baseline"):

    # Model
    simple_model = LinearRegression()
    simple_model.fit(X_train, y_train)
    y_pred = simple_model.predict(X_test)

    # Metrics
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    r2 = r2_score(y_test, y_pred)

    # Params (include requested fields)
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("solver", "N/A")
    mlflow.log_param("regularization", "none")
    mlflow.log_param("random_state", 42)

    # Metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    # Artifact: residual plot
    os.makedirs("artifacts", exist_ok=True)
    residuals = y_test - y_pred
    plt.figure(figsize=(6,4))
    plt.scatter(y_pred, residuals, alpha=0.7)
    plt.axhline(0, color="red", linestyle="--")
    plt.xlabel("Predicted")
    plt.ylabel("Residual")
    plt.title("Linear Regression Residual Plot")
    residual_plot_path = "artifacts/linear_residual_plot.png"
    plt.savefig(residual_plot_path, bbox_inches="tight")
    plt.close()
    # Takes the local file path ("artifacts/linear_residual_plot.png") and uploads that file to the MLflow server. 
    # You will be able to view this image directly in the MLflow UI for this run.
    mlflow.log_artifact(residual_plot_path)

    # Log model
    # Serializes (pickles) the Scikit-learn model and uploads it to MLflow.
    # Why do this? This allows you to "load" this exact model later for inference using mlflow.pyfunc.load_model() without needing to retrain it.
    mlflow.sklearn.log_model(simple_model, artifact_path="simple_model")

    print("Linear run logged. RMSE:", rmse, "R2:", r2)


2026/02/21 10:31:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Linear run logged. RMSE: 53.85344583676593 R2: 0.4526027629719195
